In [16]:
import tensorflow as tf
import pandas as pd

In [17]:
def custom_loss_function(y_true, y_pred):    
    y_diff = tf.abs(tf.math.log(1+y_true) - tf.math.log(1+y_pred))    
    return tf.reduce_sum(tf.square(y_diff)) / tf.cast(tf.size(y_diff), tf.float32)

In [18]:
# Load the model
model = tf.keras.models.load_model('Model1.h5', custom_objects={'custom_loss_function': custom_loss_function})
model2 = tf.keras.models.load_model('Model2.h5', custom_objects={'custom_loss_function': custom_loss_function})

In [19]:
# Load mappings pickle
import pickle
with open('mappings.pkl', 'rb') as f:
    mappings = pickle.load(f)

In [20]:
sample_data = {
    'Suburb': 'Abbotsford',
    'Rooms': 2,
    'Type': 'h',
    'Method': 'S',
    'SellerG': 'Biggin',
    'Date': '3/12/2016',
    'Distance': 2.5,
    'Postcode': 3067.0,
    'Bedroom2': 2.0,
    'Bathroom': 1.0,
    'Car': 1.0,
    'Landsize': 202.0,
    'BuildingArea': 0.0,
    'YearBuilt': 0.0,
    'CouncilArea': 'Yarra',
    'Regionname': 'Northern Metropolitan',
    'Propertycount': 4019.0
}
data = pd.DataFrame(sample_data, index=[0])
data


,Suburb,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Regionname,Propertycount
0,Abbotsford,2,h,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,0.0,0.0,Yarra,Northern Metropolitan,4019.0


In [21]:
# Convert Date to Year and Month
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data.drop('Date', axis=1, inplace=True)

categorical_cols = ['Suburb', 'Type', 'Method', 'SellerG', 'Regionname', 'CouncilArea', 'Postcode', 'Propertycount', 'Rooms', 'Postcode', 'Year', 'Month', 'YearBuilt']
# Replace categorical NA with -
for col in data.columns:
    if col in categorical_cols:
        if col == 'YearBuilt':
            data[col].fillna(0, inplace=True)
        else:
            data[col].fillna('-', inplace=True)
# Replace numerical NA with mean
for col in data.columns:
    if col not in categorical_cols:
        data[col].fillna(data[col].mean(), inplace=True)

# Use mappings to convert categorical to numerical
for col in data.columns:
    if col in categorical_cols:
        data[col] = data[col].map(mappings[col])

data


,Suburb,Rooms,Type,Method,SellerG,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Regionname,Propertycount,Year,Month
0,0,1,0,1,23,2.5,53,2.0,1.0,1.0,202.0,0.0,0,32,2,157,0,2


In [22]:
data.columns

Index(['Suburb', 'Rooms', 'Type', 'Method', 'SellerG', 'Distance', 'Postcode',
       'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt',
       'CouncilArea', 'Regionname', 'Propertycount', 'Year', 'Month'],
      dtype='object')

In [23]:
model2.predict(data)

1/1 [==============================] - 1s 1s/step


array([[31776266.]], dtype=float32)

In [24]:
model.predict(data)

1/1 [==============================] - 0s 49ms/step


array([[1013570.25]], dtype=float32)